<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0125onomatopea_bert_fine_turing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- filename: 2021_1222onomatopea_bert_fine_tuing.ipynb
- memo: 2022年01月22日現在，
- author: 浅川伸一 asakawa@ieee.org
- lincense: MIT

* transformers は M1 Mac では動作しない。具体的には，最適化関数 Adamw を呼び出すと halt する。
Intel Mac such as pasiphae では動作する。おそらく，最適化関数の下請けとして呼び出している C で書かれた関数が intel cpu や arm に特化したコードを使っている，あるいは，intel cpu や arm cpu でコンパイル済のライブラリを呼び出しているためではないかと思う。
M1 Mac は arm cpu なのだが，やや特殊なようで，libffi.dylib などのダイナミックライブラリなどでは，ターゲット cpu を arm64e としないといけないように見える。
詳細は未確認である。2022_0124

# このコードの概要，ねらい

- huggingface が提供する `transformers` から BERT を呼び出す。
`transformers` に登録されているモデルのうち，東北大学乾研提供の 日本語化 BERT モデルを微調整 fine-tuning する。
モデル名としては `cl-tohoku/bert-base-japanese` である。

- この日本語化された BERT モデル (BERT-MLM) を `BertForMaskedLM` (マスク化言語モデルに特化した BERT) として呼び出し，オノマトペ予測課題としみなして訓練を行うことである。

## 本コードの具体的な手順

1. 必要なライブラリを輸入 import 
2. 小野編 「オノマトペ辞典4500」の読み込み
3. 訓練済 日本語 BERT モデルの読み込み
4. 日本語 BERT モデルで提供されているトークナイザに，小野編「オノマトペ辞典」を登録
5. 訓練テキストデータ (original.csv) の読み込み
6. 小野版オノマトペ辞典の，各オノマトペ記述文に出てくるオノマトペを [MASK] で置換する
7. PyTorch の流儀に従って Dataset, DataLoader を定義する
8. 最適化関数を定義する
9. 訓練関数を定義して訓練を行う
10. 結果の損失関数の減衰曲線を描画する


---

## 1.  必要なライブラリを輸入 import 

In [1]:
import os
import sys
import numpy as np
import unicodedata
from termcolor import colored

# 本ファイルを Google Colaboratory 上で実行する場合に，必要となるライブラリをインストールする
import platform
isColab = platform.system() == 'Linux'
if isColab:
    !pip install transformers > /dev/null 2>&1 

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1
    !pip install jaconv > /dev/null 2>&1
    !pip install japanize_matplotlib > /dev/null 2>&1    

In [2]:
# PyTorch の seed の設定関連 再現性確保のため
# https://qiita.com/takubb/items/7d45ae701390912c7629
# https://qiita.com/si1242/items/d2f9195c08826d87d6ad
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# リソースの選択（CPU/GPU）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 乱数シード固定（再現性の担保）
def fix_seed(seed):
    # random
    random.seed(seed)
    # numpy
    np.random.seed(seed)
    
    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.random.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 42
fix_seed(seed)
# データローダーのサブプロセスの乱数のseedが固定
def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)
    print(worker_init_fn(1))
    
# # データローダーの作成
# train_loader = torch.utils.data.DataLoader(train_dataset,
#                                            batch_size=16,  # バッチサイズ
#                                            shuffle=True,  # データシャッフル
#                                            num_workers=2,  # 高速化
#                                            pin_memory=True,  # 高速化
#                                            worker_init_fn=worker_init_fn
#                                            )

## 2. 小野編 「オノマトペ辞典4500」の読み込み

In [4]:
# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータの読み込み

#'日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
# そのため Google Colab での解法，ローカルファイルよりアップロードしてください
if isColab:
    from google.colab import files
    uploaded = files.upload()  # ここで `日本語オノマトペ辞典4500より.xls` を指定してアップロードする
    data_dir = '.'
else:
    data_dir = '/Users/asakawa/study/2021ccap/notebooks'

import pandas as pd
import jaconv

onomatopea_excel = '日本語オノマトペ辞典4500より.xlsx'
onmtp2761 = pd.read_excel(os.path.join(data_dir, onomatopea_excel), sheet_name='2761語')

#すべてカタカナ表記にしてデータとして利用する場合
#`日本語オノマトペ辞典4500` はすべてひらがな表記だが，一般にオノマトペはカタカナ表記されることが多いはず
#onomatopea = list(sorted(set([jaconv.hira2kata(o) for o in onmtp2761['オノマトペ']])))

# Mac と Windows の表記の相違を吸収
onomatopea = list(sorted(set([jaconv.normalize(o) for o in onmtp2761['オノマトペ']])))
print(f'データファイル名: {os.path.join(data_dir, onomatopea_excel)}\n',
      f'オノマトペ単語総数: len(onomatopea):{len(onomatopea)}')

データファイル名: ./日本語オノマトペ辞典4500より.xlsx
 オノマトペ単語総数: len(onomatopea):1740


## 3. 訓練済 日本語 BERT モデルの読み込みと，小野編「オノマトペ辞典」のトークナイザへの登録

In [ ]:
# transformers, huggingface 版の BERT 実装の読み込み
import torch
from transformers import BertConfig
from transformers import BertForPreTraining
from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM

model_ja_name = 'cl-tohoku/bert-base-japanese'  # 東北大学乾研による 日本語 BERT 実装
model = BertForMaskedLM.from_pretrained(model_ja_name) # マスク化言語モデルを指定
config = BertConfig.from_pretrained(model_ja_name)

# GPU が利用可能であれば利用する
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

tknz1 = BertJapaneseTokenizer.from_pretrained(model_ja_name)
# BPE (or sentencepiece) による下位単語分割あり

In [23]:
# transformers, huggingface 版の BERT 実装の読み込み
import torch
from transformers import BertConfig
from transformers import BertForPreTraining
from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM

#model_ja_name = 'cl-tohoku/bert-base-japanese'  # 東北大学乾研による 日本語 BERT 実装

# see https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2
model_ja_name = 'sonoisa/sentence-bert-base-ja-mean-tokens-v2'  # 東北大学乾研による 日本語 BERT 実装
model = BertForMaskedLM.from_pretrained(model_ja_name) # マスク化言語モデルを指定
config = BertConfig.from_pretrained(model_ja_name)

# GPU が利用可能であれば利用する
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

tknz1 = BertJapaneseTokenizer.from_pretrained(model_ja_name)
# BPE (or sentencepiece) による下位単語分割あり

Downloading:   0%|          | 0.00/667 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of BertForMaskedLM were not initialized from the model checkpoint at sonoisa/sentence-bert-base-ja-mean-tokens-v2 and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

## 4. 日本語 BERT モデルで提供されているトークナイザに，小野編「オノマトペ辞典」を登録


In [24]:
# トークナイザ の修正，実際には onomatopea 単語リストを引数に指定して `add_tokens()` を呼び出すだけ
# ただし，語彙数 tknz.vocab は変更されない。追加された語彙，本コードの場合はオノマトペは，
# `tknz1.added_tokens_encoder` と `tknz1.added_tokens_decoder` に反映されているためである
num_added = tknz1.add_tokens(onomatopea)
print(f'追加されたトークン数:{num_added}/オノマトペ数:{len(onomatopea)}') 
model.resize_token_embeddings(len(tknz1))

print(f' len(tknz1):{len(tknz1)}\n', 
      f'len(tknz1.vocab):{len(tknz1.vocab)}\n',  # 一見すると，この数字からオノマトペが追加されていないように見える。
      f'tknz1.vocab_size:{tknz1.vocab_size}')    # 駄菓子菓子，下で見るように，正しく動作しているように見受けられる

print('# 確認用')
for w in onomatopea[-5:]:
    idx = tknz1.convert_tokens_to_ids(w)
    w_ = tknz1.convert_ids_to_tokens(idx)
    print(f'単語:{w}(id:{idx}) -> token:{w_}')
    

追加されたトークン数:1710/オノマトペ数:1740
 len(tknz1):33710
 len(tknz1.vocab):32000
 tknz1.vocab_size:32000
# 確認用
単語:わんさわんさ(id:33705) -> token:わんさわんさ
単語:わんわ(id:33706) -> token:わんわ
単語:わんわん(id:33707) -> token:わんわん
単語:わーっ(id:33708) -> token:わーっ
単語:わーわー(id:33709) -> token:わーわー


## 5. 訓練テキストデータ (original.csv) の読み込み

In [25]:
# 近藤先生 (2021年12月22日） から送っていただいた，オノマトペ文章データ 'original.csv' を読み込む
import jaconv

if isColab:
    uploaded = files.upload()  # original.csv をアップロード
    data_dir = '.'
else:
    data_dir = '/Users/asakawa/study/2021kondo_project'

original = []
n = 0
with open(os.path.join(data_dir,'original.csv'), 'r', encoding='utf8') as f:
    s = f.read()
    for s_ in s.split('\n'):
        if n == 0:
            n += 1
            continue
        idx, sent = s_.split(',')
        
        # Mac と Windows との unicode 符号化の差分を吸収する
        # jaconv.normalize は内部で unicodedata.normalize('NFKC') を呼び出しているので
        # 差異 between Mac and Windows を吸収できる
        sent = ''.join(jaconv.normalize(x) for x in sent)
        original.append(sent)
        #original[int(idx)] = sent

print(f'{len(original)} has been read')

2469 has been read


## 6. 小野版オノマトペ辞典の，各オノマトペ記述文に出てくるオノマトペを [MASK] で置換する

In [26]:
max_token_len = np.array([len(tknz1(s).input_ids) for s in original]).max()
max_token_len += 2  # 保険のため 2 くらい加えておく
print(f'max_token_len:{max_token_len}')

# トークナイザにかけて出力を得る。`max_length` のデフォルトは 512 だが，今回は長文である必要がないと考えられる。
# ここでは `max_token_len = 23` にしている。512 でも動作するが，学習に要する時間が増える
text = tuple(original)  # 全文をタプルに変換
inputs = tknz1(text, 
               return_tensors='pt', 
               max_length=max_token_len, 
               truncation=True, 
               padding='max_length')

#`labels` キーを追加する。実際には inputs_ids なのでラベルではなくトークンID の系列
inputs['labels'] = inputs.input_ids.detach().clone()

#トークン ID を走査して，オノマトペ単語であれば，[MASK] トークンに置き換える。
l_ = []
for l in inputs['labels']:
    l_.append([tknz1.mask_token_id if w in onomatopea else tknz1.convert_tokens_to_ids(w) for w in tknz1.convert_ids_to_tokens(l)])

inputs['input_ids'] = torch.LongTensor(l_)
#print(inputs['input_ids'].shape)

max_token_len:23


## 7. PyTorch の流儀に従って Dataset, DataLoader を定義する


In [27]:
#データセットのためのクラスを定義
class onmtpDataset(torch.utils.data.Dataset):
    def __init__(self, encoder):
        self.encoder = encoder
        
    def __getitem__(self, idx):
        return {key:torch.tensor(val[idx]) for key, val in self.encoder.items()}
    
    def __len__(self):
        return len(self.encoder.input_ids)
    
dataset = onmtpDataset(inputs)

#データローダを準備
loader = torch.utils.data.DataLoader(dataset, 
                                     batch_size=8, 
                                     shuffle=True,
                                     worker_init_fn=worker_init_fn
                                     )

# GPU/CPU 使用を設定し，モデルの訓練モードを起動 #Setup GPU/CPU usage and activate the training mode of our model.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# そしてモデルを選択したデバイスに移動 # and move our model over to the selected device
model.to(device)
# 訓練モードに設定 #activate training mode
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(33710, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
           

## 8. 最適化関数を定義する

In [28]:
from torch.optim import AdamW

#最適化関数を初期化 # initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

## 9. 訓練関数を定義して訓練を行う

In [ ]:
from tqdm import tqdm  # 進捗状況の可視化のため

epochs = 50  # 学習回数を指定

losses = []
for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:

        optim.zero_grad()  # 学習に用いる BERT パラメータの勾配を 0 で初期化

        input_ids = batch['input_ids'].to(device)  # ミニバッチサイズだけデータを取得
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # BERT を呼び出して結果を得る
        outputs = model(input_ids, 
                        attention_mask=attention_mask,
                        labels=labels)

        loss = outputs.loss  # 損失値を取得
        loss.backward()      # 取得した損失値に基づいて BERT のパラメータを逆伝播
        optim.step()         # BERT パラメータの更新 すなわち学習
        
        loop.set_description(f'Epoch {epoch}') # 進行状況の表示
        loop.set_postfix(loss=loss.item())
        losses.append(loss.item())

## 10. 結果の損失関数の減衰曲線を描画する

In [ ]:
len(losses)
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.plot(losses[:])
plt.xlabel('訓練時間')
plt.ylabel('損失値')
plt.title('オノマトペ微調整における学習の推移 (損失値) の減少')
plt.show()

In [14]:
uploaded = files.upload()  # ここで学習済みファイル `*.pt` を指定してアップロードする

Saving 2022_0125onomatopea.pt to 2022_0125onomatopea.pt


In [ ]:
out_fname = '2022_0125onomatopea.pt'
#torch.save(model.state_dict(), out_fname)
#model = BertForMaskedLM.from_pretrained(model_ja_name)
model.load_state_dict(torch.load(out_fname))
files.download(out_fname)

In [ ]:
model.to(device)
model.eval()

def eval_an_output(N, original=original, tknz1=tknz1, inputs=inputs, print_flag=True):
    """
    引数として 数字を 1 つ入力すると (N)，`original.csv` の N 行目のデータを読み込んで，
    その文のオノマトペを [MASK] に置き換えて，マスク化言語モデルで [MASK] を予測する。
    結果を表示する場合には 引数 `print_flag=True` として呼び出す
    """
    if N >= len(original) or (not isinstance(N, int)):
        return

    _out = model(inputs.input_ids[N].unsqueeze(0).to(device), attention_mask=inputs.attention_mask[N].unsqueeze(0).to(device), labels=inputs.labels[N].to(device))
    _x = _out.logits.detach()
    __x = _x.squeeze(0).detach().clone()
    _pred_idx  = torch.argmax(__x, dim=1, keepdim=True)
    _pred_s    = "/".join(tknz1.convert_ids_to_tokens(_pred_idx)).replace('/[PAD]','')
    
    _orig      = original[N] # 原文
    _inp_idx   = tknz1.convert_ids_to_tokens(inputs.input_ids[N]) # 入力トークンID
    _inp_s     = "/".join(_inp_idx).replace('/[PAD]','')          # 入力文
    _teach_idx = tknz1.convert_ids_to_tokens(inputs.labels[N])    # 教師信号トークンID
    _teach_s   = "/".join(_teach_idx).replace('/[PAD]','')        # 教師信号文
    
    _mask_pos = np.where(inputs.input_ids[N].detach().numpy() == tknz1.mask_token_id)
    _teach_tokens = inputs.labels[N][_mask_pos].detach().squeeze().numpy()
    _pred_tokens  = _pred_idx[_mask_pos].detach().squeeze().cpu().numpy()

    _n_hit = np.array([_teach_tokens == _pred_tokens]).sum()       # 正解したか否か
    if print_flag:
        color = 'grey' if _n_hit > 0 else 'red'
        print(f'{N:5,d}   原文:{_orig}')
        print(f'\t入力:{_inp_s}')
        print(f'\t正解:{_teach_s}')
        print(colored(f'\t出力:{_pred_s}',color))
        print(f'\tmask 位置:{_mask_pos}')
        print(f'\t正解トークン:{_teach_tokens}', f'予測トークン:{_pred_tokens}', 
              f'{np.array([_teach_tokens == _pred_tokens]).sum() > 0}')
        print(f'\t_out.loss:{_out.loss:.3f}')
    
    return _out.loss, _n_hit

total_hit = 0
for i in range(len(original)):
    _, hit = eval_an_output(i, print_flag=False) # print_flag = True にすると推論結果を表示します. 逆に False にすれば正解率だけ計算します
    total_hit += hit

print(f'正解数:{total_hit}/{len(original)}= {total_hit/len(original) * 100:.3f} %')    

In [ ]:
out_fname = '2022_0125onomatopea_mlm_finetuned.pt'
#torch.save(model.state_dict(), out_fname)
#model = BertForMaskedLM.from_pretrained(model_ja_name)
model.load_state_dict(torch.load(out_fname))
files.download(out_fname)

In [ ]:
!cp /content/2022_0125onomatopea_mlm_finetuned.pt drive/MyDrive